In [63]:
# paths 

# secrets file
secrets_path = r"C:\Users\localadmin\Desktop\new desktop\linkedinsucks\user_files\secrets.json"
persistent_storage_path = r"C:\Users\localadmin\Desktop\new desktop\linkedinsucks\linkedin_easyapply_bot\all_data\persistent_storage_index"
docs_path = r"C:\Users\localadmin\Desktop\new desktop\linkedinsucks\linkedin_easyapply_bot\all_data\docs"

In [64]:
import json
import os
import torch
from llama_index.core import Settings, VectorStoreIndex, SummaryIndex, SimpleDirectoryReader
from llama_index.llms.ollama import Ollama
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.tools import QueryEngineTool
from llama_index.core.query_engine.router_query_engine import RouterQueryEngine
from llama_index.core.selectors import LLMSingleSelector





# Check if CUDA is available
if torch.cuda.is_available():
    # Get the name of the GPU
    gpu_name = torch.cuda.get_device_name(torch.cuda.current_device())
    print(f"Using GPU: {gpu_name}")
else:
    print("CUDA is not available. Using CPU.")


with open(secrets_path, 'r') as file:
    data = json.load(file)
    

OPENAI_API_KEY = data["OPENAI_KEY"]
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

Using GPU: NVIDIA GeForce GTX 1650


In [66]:
system_prompt =  """
answer questions about lectures
"""
#### Define LLMs and Embeddings for Each Tool

# getting necessaery language models 
gpt3_5_llm = OpenAI(model="gpt-3.5-turbo")
gpt3_5_embed = OpenAIEmbedding(model="text-embedding-ada-002")
system_prompt =  """
answer questions about lectures
"""


# For vector search tool (Using phi3-mini)
phi3_llm = Ollama(
    model="phi3:3.8b-mini-4k-instruct-q4_K_M",
    temperature=0.01,
    request_timeout=400,
    system_prompt=system_prompt,  # Define your system prompt as needed
    context_window=2000
)
phi3_embed = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")

# tl_llm = Ollama(
# model = "tinyllama",
# temperature = 0.01,
# request_timeout = 400,
# system_prompt = system_prompt,
# context_window = 2000

# )
# tl_embed = HuggingFaceEmbedding(
#     model_name="sentence-transformers/all-MiniLM-L6-v2"
# )

In [74]:
 system_prompt: Optional[str] = None,
    messages_to_prompt: Annotated[Optional[llama_index.core.llms.llm.MessagesToPromptType], WithJsonSchema(json_schema={'type': 'string'}, mode=None)] = None,
    completion_to_prompt: Annotated[Optional[llama_index.core.llms.llm.CompletionToPromptType], WithJsonSchema(json_schema={'type': 'string'}, mode=None)] = None,

    query_wrapper_prompt: Optional[llama_index.core.prompts.base.BasePromptTemplate]

Init signature:
Ollama(
    model: str,
    base_url: str = 'http://localhost:11434',
    temperature: float = 0.75,
    context_window: int = 3900,
    request_timeout: float = 30.0,
    prompt_key: str = 'prompt',
    json_mode: bool = False,
    additional_kwargs: Dict[str, Any] = {},
    client: Optional[ollama._client.Client] = None,
    async_client: Optional[ollama._client.AsyncClient] = None,
    is_function_calling_model: bool = True,
    *,
    callback_manager: llama_index.core.callbacks.base.CallbackManager = None,
    system_prompt: Optional[str] = None,
    messages_to_prompt: Annotated[Optional[llama_index.core.llms.llm.MessagesToPromptType], WithJsonSchema(json_schema={'type': 'string'}, mode=None)] = None,
    completion_to_prompt: Annotated[Optional[llama_index.core.llms.llm.CompletionToPromptType], WithJsonSchema(json_schema={'type': 'string'}, mode=None)] = None,
    output_parser: Optional[llama_index.core.types.BaseOutputParser] = None,
    pydantic_program_mode: 

In [ ]:
short_qna_prompt = """
you are a job application bot that is applying for jobs on behalf of the given profile. 
answer questions carefully and accurately based on the given context and documents. 
For questions beginning with how many, always answer in integer only.
For questions asking number of years of experience, if you do not find any relevant information in the documents answer 1.
your goal is to get a job by answering to the most optimal extent. 
maximise results by optimizing to give good responses to the application
"""

short_prompt_wrapper = """"

template="answer questions concisely and accurately: {query}",
    input_variables=["query"]
"""

In [67]:

 
system_prompt =  """
answer questions about lectures
"""

gpt3_5_llm = OpenAI(model="gpt-3.5-turbo")
gpt3_5_embed = OpenAIEmbedding(model="text-embedding-ada-002")
system_prompt =  """
answer questions about lectures
"""


# For vector search tool (Using phi3-mini)
phi3_llm = Ollama(
    model="phi3:3.8b-mini-4k-instruct-q4_K_M",
    temperature=0.01,
    request_timeout=400,
    system_prompt=system_prompt,  # Define your system prompt as needed
    context_window=2000
)
phi3_embed = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")




# Function to get the summarization tool using GPT-3.5
def get_summary_tool(docs):
    # Split documents into chunks
    splitter = SentenceSplitter(chunk_size=1024)
    documents = splitter.get_nodes_from_documents(docs)


    # Set LLM and embedding model for the summarizer tool
    Settings.llm = gpt3_5_llm
    Settings.embed_model = gpt3_5_embed

    # Create SummaryIndex and QueryEngine for summarization
    summary_index = SummaryIndex(documents)
    summary_query_engine = summary_index.as_query_engine(
        response_mode="tree_summarize",
        use_async=True,
    )

    # Create and return the summary tool
    summary_tool = QueryEngineTool.from_defaults(
        name="summary_tool",
        query_engine=summary_query_engine,
        description="Useful for summarization questions related to the documents."
    )

    return summary_tool

# Function to get the vector search tool using phi3-mini
def get_vector_tool(docs):
    # Split documents into chunks
    splitter = SentenceSplitter(chunk_size=1024)
    documents = splitter.get_nodes_from_documents(docs)

    # Set LLM and embedding model for the vector search tool
    Settings.llm = phi3_llm
    Settings.embed_model = phi3_embed

    # Create VectorStoreIndex and QueryEngine for vector search
    vector_index = VectorStoreIndex(documents)
    vector_query_engine = vector_index.as_query_engine()

    # Create and return the vector tool
    vector_tool = QueryEngineTool.from_defaults(
        name="vector_tool",
        query_engine=vector_query_engine,
        description="Useful for retrieving specific context from the documents."
    )

    return vector_tool

# Example usage:
# Assuming `docs` is the data you want to process
docs = SimpleDirectoryReader(docs_path).load_data()

# Get the summarization and vector tools
summary_tool = get_summary_tool(docs)
vector_tool = get_vector_tool(docs)

# Now you can use summary_tool and vector_tool for your queries


query_engine = RouterQueryEngine(
    selector=LLMSingleSelector.from_defaults(),
    query_engine_tools=[
        summary_tool,
        vector_tool,
    ],
    verbose=True
)


In [68]:
docs = SimpleDirectoryReader(docs_path).load_data()

In [69]:



query_engine = RouterQueryEngine(
    selector=LLMSingleSelector.from_defaults(),
    query_engine_tools=[
        summary_tool,
        vector_tool,
    ],
    verbose=True
)

In [71]:
response = query_engine.query("how many years of experience does he have in python")
print(str(response))

Selecting query engine 1: The question 'how many years of experience does he have in python' is asking to retrieve specific context from the documents regarding an individual’s professional background..
5 years
